In [1]:
# Installation of tangelo if not already installed.
try:
    import tangelo
except ModuleNotFoundError:
    !pip install git+https://github.com/goodchemistryco/Tangelo.git@develop --quiet
    !pip install qulacs pyscf --quiet

# Import for a pretty jupyter notebook.
import json

# Molecule definition.
from tangelo import SecondQuantizedMolecule as SQMol

# The minimal import foRr DMET.
from tangelo.problem_decomposition import DMETProblemDecomposition
# Ability to change localization method.
from tangelo.problem_decomposition.dmet import Localization
# Use for VQE ressources estimation vs DMET.
from tangelo.algorithms import VQESolver
# Use for comparison.
from tangelo.algorithms import FCISolver

In [2]:
import numpy as np
from tangelo.toolboxes.ansatz_generator.ansatz import Ansatz
from tangelo.toolboxes.qubit_mappings.statevector_mapping import get_reference_circuit
from tangelo.toolboxes.qubit_mappings.mapping_transform import get_qubit_number, fermion_to_qubit_mapping
from tangelo.linq import Circuit, Gate

# DMET-VQE for excited state energy of LiH

I decomposed the molecule into Li and H for this one.

In [3]:
LiH = [('Li', (0, 0, 0)),('H', (0, 0, 1.6))]

fo = [0]

# Runs RHF calculation
mol_LiH = SQMol(LiH, q=0, spin=0, basis='minao', frozen_orbitals=fo, symmetry=True) #or use basis cc-pVDZ

# Runs ROHF calculation
mol_LiH_t = SQMol(LiH, q=0, spin=2, basis="minao", frozen_orbitals=fo, symmetry=True)

In [4]:
# Symmetry labels and occupations for frozen core and active orbitals
print("  #  Energy  Symm Occ")
for i in range(3):
    print(f"{i+1:3d}{mol_LiH.mo_energies[i]: 9.4f}  {mol_LiH.mo_symm_labels[i]}   {int(mol_LiH.mo_occ[i])}")

# Active electrons, Active orbitals
print(f"Number of active electrons: {mol_LiH.n_active_electrons}")
print(f"Number of active orbtials: {mol_LiH.n_active_mos}")

  #  Energy  Symm Occ
  1  -2.4686  A1   2
  2  -0.2945  A1   2
  3   0.0909  A1   0
Number of active electrons: 2
Number of active orbtials: 2


In [5]:
print(mol_LiH.symmetry)

True


In [6]:
options_LiH_dmet = {"molecule": mol_LiH,
                    "fragment_atoms": [1,1],
                    "fragment_solvers": "vqe",
                    "verbose": False
                    }

dmet_LiH = DMETProblemDecomposition(options_LiH_dmet)
dmet_LiH.build()

In [7]:
resources_LiH_dmet = dmet_LiH.get_resources()
print(resources_LiH_dmet[0])

{'qubit_hamiltonian_terms': 118, 'circuit_width': 6, 'circuit_depth': 385, 'circuit_2qubit_gates': 272, 'circuit_var_gates': 40, 'vqe_variational_parameters': 5}


In [8]:
fci_LiH = FCISolver(mol_LiH)
energy_LiH_fci = fci_LiH.simulate()

print(f"FCI energy (hartree): \t {energy_LiH_fci}")

FCI energy (hartree): 	 -7.979986343677684


In [9]:
from tangelo.algorithms.variational import VQESolver, BuiltInAnsatze
from tangelo.algorithms.classical import FCISolver

# Dictionary of resources for each algorithm
algorithm_resources = dict()

# Ground state energy calculation with VQE, reference values with FCI
vqe_options = {"molecule": mol_LiH, "ansatz": BuiltInAnsatze.UCCSD}
vqe_solver = VQESolver(vqe_options)
vqe_solver.build()
vqe_energy = vqe_solver.simulate()
print("\n Ground Singlet state")
print(f"VQE energy = {vqe_energy}")
print(f"CASCI energy = {FCISolver(mol_LiH).simulate()}")
algorithm_resources["vqe_ground_state"] = vqe_solver.get_resources()


 Ground Singlet state
VQE energy = -7.979986033529784
CASCI energy = -7.979986343677684


In [10]:
# First excited state energy calculation with VQE, reference values with FCI
vqe_options = {"molecule": mol_LiH_t, "ansatz": BuiltInAnsatze.UpCCGSD}
vqe_solver_t = VQESolver(vqe_options)
vqe_solver_t.build()
vqe_energy_t = vqe_solver_t.simulate()
print("\n Lowest Triplet state")
print(f"VQE energy = {vqe_energy_t}")
print(f"CASCI energy = {FCISolver(mol_LiH_t).simulate()}")
algorithm_resources["vqe_triplet_state"] = vqe_solver_t.get_resources()


 Lowest Triplet state
VQE energy = -7.877228340299112
CASCI energy = -7.877228340299247


In [11]:
# Add initial VQE optimal circuit to the deflation circuits list
deflation_circuits = [vqe_solver.optimal_circuit.copy()]

# Calculate first and second excited states by adding optimal circuits to deflation_circuits
for i in range(2):
    vqe_options = {"molecule": mol_LiH, "ansatz": BuiltInAnsatze.UpCCGSD, 
                   "deflation_circuits": deflation_circuits, "deflation_coeff": 0.4}
    vqe_solver = VQESolver(vqe_options)
    vqe_solver.build()
    vqe_energy = vqe_solver.simulate()
    print(f"Excited state #{i+1} \t VQE energy = {vqe_energy}")
    algorithm_resources[f"vqe_deflation_state_{i+1}"] = vqe_solver.get_resources()

    deflation_circuits.append(vqe_solver.optimal_circuit.copy())

Excited state #1 	 VQE energy = -7.752189149469342
Excited state #2 	 VQE energy = -7.5799825127814735


In [12]:
vqe_options = {"molecule": mol_LiH, "ansatz": BuiltInAnsatze.UpCCGSD, 
               "deflation_circuits": deflation_circuits,
               "deflation_coeff": 0.4, "ref_state": [1, 0, 1, 0]}
vqe_solver_triplet = VQESolver(vqe_options)
vqe_solver_triplet.build()
vqe_energy = vqe_solver_triplet.simulate()
print(f"VQE energy = {vqe_energy}")
algorithm_resources[f"vqe_deflation_state_{3}"] = vqe_solver_triplet.get_resources()

VQE energy = -7.877227970021433


Classical Result:

In [13]:
from pyscf import mcscf

# Hartree-Fock calculation (assuming `mol_LiH` is already defined)
myhf = mol_LiH.mean_field

# Define the core and active orbitals using their symmetries
ncore = {"A1": 1}  # Freeze the first A1 orbital (fully occupied, core orbital)
ncas = {"A1": 2}   # Use the remaining 2 orbitals in the A1 symmetry as active orbitals

# Setup and run CASCI calculation
print("Calculation for A1 symmetry")

# Setup CASCI with 2 active electrons in 2 active orbitals (both in A1 symmetry)
mc = mcscf.CASCI(myhf, 2, (1, 1))

# Sort the molecular orbitals by their irreps
mo = mc.sort_mo_by_irrep(cas_irrep_nocc=ncas, cas_irrep_ncore=ncore)

# Run the CASCI calculation
mc.fcisolver.wfnsym = "A1"  # Set the wavefunction symmetry to A1
mc.fcisolver.nroots = 2  # Calculate both the ground and first excited states
emc_A1 = mc.casci(mo)[0]

# Print the result for A1 symmetry
print(f"Energy for A1 symmetry: {emc_A1} Eh")

Calculation for A1 symmetry

WARN: Mulitple states found in CASCI solver. First state is used to compute the Fock matrix and natural orbitals in active space.

CASCI state   0  E = -7.97998634367768  E(CI) = -1.07431109981599  S^2 = 0.0000000
CASCI state   1  E = -7.87722797002157  E(CI) = -0.971552726159873  S^2 = 2.0000000
Energy for A1 symmetry: [-7.97998634 -7.87722797] Eh


/Users/rohan/.pyenv/versions/3.12.5/envs/nyc_haq/lib/python3.12/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/rohan/.pyenv/versions/3.12.5/envs/nyc_haq/lib/python3.12/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


In [14]:
import numpy as np
from scipy.linalg import eigh
from openfermion.utils import hermitian_conjugated as hc

from tangelo.toolboxes.operators import FermionOperator
from tangelo.toolboxes.qubit_mappings.mapping_transform import fermion_to_qubit_mapping as f2q_mapping

# Generate all single excitations as qubit operators
op_list = list()
for i in range(2):
    for j in range(i+1, 2):
        op_list += [f2q_mapping(FermionOperator(((2*i, 1), (2*j, 0))), "jw")] #spin-up transition
        op_list += [f2q_mapping(FermionOperator(((2*i+1, 1), (2*j+1, 0))), "jw")] #spin-down transition
        op_list += [f2q_mapping(FermionOperator(((2*i+1, 1), (2*j, 0))), "jw")] #spin-up to spin-down
        op_list += [f2q_mapping(FermionOperator(((2*i, 1), (2*j+1, 0))), "jw")] #spin-down to spin-up

# Compute F and S matrices.
size_mat = len(op_list)
h = np.zeros((size_mat, size_mat))
s = np.zeros((size_mat, size_mat))
state_circuit = vqe_solver.optimal_circuit
for i, op1 in enumerate(op_list):
    for j, op2 in enumerate(op_list):
        h[i, j] = np.real(vqe_solver.backend.get_expectation_value(hc(op1)*vqe_solver.qubit_hamiltonian*op2, state_circuit))
        s[i, j] = np.real(vqe_solver.backend.get_expectation_value(hc(op1)*op2, state_circuit))

label = "quantum_subspace_expansion"
algorithm_resources[label] = vqe_solver.get_resources()
algorithm_resources[label]["n_post_terms"] = len(op_list)**2*algorithm_resources[label]["qubit_hamiltonian_terms"]

In [15]:
# Solve FU = SUE
e, v = eigh(h,s)
print(f"Quantum Subspace Expansion energies: \n {e}")

Quantum Subspace Expansion energies: 
 [-7.87722797 -7.87722797 -7.87722797 -7.74170155]


# Using VQE solely to get excited state of LiH

In [16]:
# non-decompose case complexity:

LiH_noD = """
Li 0.  0. 0.
H  0.  0. 1.6
"""

fo = [0]

# Runs RHF calculation
mol_LiH_noD = SQMol(LiH_noD, q=0, spin=0, basis='minao', frozen_orbitals=fo, symmetry=True)
# Runs ROHF calculation
mol_LiH_noD_t = SQMol(LiH_noD, q=0, spin=2, basis="minao", frozen_orbitals=fo, symmetry=True)

options_LiH_noD_vqe = {"molecule": mol_LiH_noD, "qubit_mapping": "jw", "verbose": False}
vqe_LiH_noD = VQESolver(options_LiH_noD_vqe)
vqe_LiH_noD.build()

resources_LiH_noD_vqe = vqe_LiH_noD.get_resources()
print(resources_LiH_noD_vqe)

{'qubit_hamiltonian_terms': 27, 'circuit_width': 4, 'circuit_depth': 100, 'circuit_2qubit_gates': 64, 'circuit_var_gates': 12, 'vqe_variational_parameters': 2}


In [17]:
# Ground state energy calculation with VQE, reference values with FCI
vqe_energy = vqe_LiH_noD.simulate()
print("\n Ground Singlet state")
print(f"VQE energy = {vqe_energy}")
print(f"CASCI energy = {FCISolver(mol_LiH_noD).simulate()}")
algorithm_resources["vqe_ground_state"] = vqe_LiH_noD.get_resources()

# First excited state energy calculation with VQE, reference values with FCI
vqe_options = {"molecule": mol_LiH_noD_t, "ansatz": BuiltInAnsatze.UpCCGSD}
vqe_solver_t = VQESolver(vqe_options)
vqe_solver_t.build()
vqe_energy_t = vqe_solver_t.simulate()
print("\n Lowest Triplet state")
print(f"VQE energy = {vqe_energy_t}")
print(f"CASCI energy = {FCISolver(mol_LiH_noD_t).simulate()}")
algorithm_resources["vqe_triplet_state"] = vqe_solver_t.get_resources()


 Ground Singlet state
VQE energy = -7.979986033529784
CASCI energy = -7.979986343677684

 Lowest Triplet state
VQE energy = -7.877228340299112
CASCI energy = -7.877228340299247


In [18]:
# Add initial VQE optimal circuit to the deflation circuits list
deflation_circuits = [vqe_solver.optimal_circuit.copy()]

# Calculate first and second excited states by adding optimal circuits to deflation_circuits
for i in range(2):
    vqe_options = {"molecule": mol_LiH_noD, "ansatz": BuiltInAnsatze.UpCCGSD, 
                   "deflation_circuits": deflation_circuits, "deflation_coeff": 0.4}
    vqe_solver = VQESolver(vqe_options)
    vqe_solver.build()
    vqe_energy = vqe_solver.simulate()
    print(f"Excited state #{i+1} \t VQE energy = {vqe_energy}")
    algorithm_resources[f"vqe_deflation_state_{i+1}"] = vqe_solver.get_resources()

    deflation_circuits.append(vqe_solver.optimal_circuit.copy())

Excited state #1 	 VQE energy = -7.752200465710246
Excited state #2 	 VQE energy = -7.580002948358137


In [19]:
vqe_options = {"molecule": mol_LiH_noD, "ansatz": BuiltInAnsatze.UpCCGSD, 
               "deflation_circuits": deflation_circuits,
               "deflation_coeff": 0.4, "ref_state": [1, 0, 1, 0]}
vqe_solver_triplet = VQESolver(vqe_options)
vqe_solver_triplet.build()
vqe_energy = vqe_solver_triplet.simulate()
print(f"VQE energy = {vqe_energy}")
algorithm_resources[f"vqe_deflation_state_{3}"] = vqe_solver_triplet.get_resources()

VQE energy = -7.877227970021433


# Choosing the basis affect the complexity of the circuit and the final result. So, here is the sample generator to test all those basis with DMET-VQE and VQE.

In [20]:
LiH_tester = """
Li 0.  0. 0.
H  0.  0. 1.6
"""

fo = [0]

# Runs ROHF calculation
mol_LiH_tester = SQMol(LiH_tester, q=0, spin=2, basis="minao", frozen_orbitals=fo, symmetry=True)

options_LiH_tester_vqe = {"molecule": mol_LiH_tester, "qubit_mapping": "jw", "verbose": False}
vqe_LiH_tester = VQESolver(options_LiH_tester_vqe)
vqe_LiH_tester.build()

resources_LiH_tester_vqe = vqe_LiH_tester.get_resources()
print(resources_LiH_tester_vqe)

{'qubit_hamiltonian_terms': 23, 'circuit_width': 4, 'circuit_depth': 1, 'circuit_2qubit_gates': 0, 'circuit_var_gates': 0, 'vqe_variational_parameters': 0}


/Users/rohan/.pyenv/versions/3.12.5/envs/nyc_haq/lib/python3.12/site-packages/tangelo/algorithms/variational/vqe_solver.py:243: RuntimeWarning: No variational gate found in the circuit.
  warnings.warn("No variational gate found in the circuit.", RuntimeWarning)
